<a href="https://colab.research.google.com/github/krischmi/projects/blob/main/Airport_Data_KS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests

In [2]:
import requests

url = "https://aerodatabox.p.rapidapi.com/airports/search/location/52.3086/4.763889/km/10/16"

querystring = {"withFlightInfoOnly":"true"}

headers = {
	"X-RapidAPI-Key": "fc1e58f82cmsh80f890b0cc58cfep1656acjsnb3d61b0144ed",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"searchBy":{"lat":52.3086,"lon":4.763889},"items":[{"icao":"EHAM","iata":"AMS","name":"Amsterdam, Amsterdam Schiphol","shortName":"Schiphol","municipalityName":"Amsterdam","location":{"lat":52.3086,"lon":4.763889},"countryCode":"NL"}]}


In [3]:
response.json()

{'searchBy': {'lat': 52.3086, 'lon': 4.763889},
 'items': [{'icao': 'EHAM',
   'iata': 'AMS',
   'name': 'Amsterdam, Amsterdam Schiphol',
   'shortName': 'Schiphol',
   'municipalityName': 'Amsterdam',
   'location': {'lat': 52.3086, 'lon': 4.763889},
   'countryCode': 'NL'}]}

In [4]:
df=pd.json_normalize(response.json())
df

,items,searchBy.lat,searchBy.lon
0,"[{'icao': 'EHAM', 'iata': 'AMS', 'name': 'Amst...",52.3086,4.763889


In [5]:
new_df=pd.DataFrame.from_dict(df["items"][0])
new_df

,icao,iata,name,shortName,municipalityName,location,countryCode
0,EHAM,AMS,"Amsterdam, Amsterdam Schiphol",Schiphol,Amsterdam,"{'lat': 52.3086, 'lon': 4.763889}",NL


In [6]:
def icao_airport_codes(latitudes, longitudes):

  #assert len(latitudes) == len(longitudes)
  
  list_for_df = []

  for i in range(len(latitudes)):

    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{latitudes[i]}/{longitudes[i]}/km/20/16"

    querystring = {"withFlightInfoOnly":"true"}

    headers = {
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
      "X-RapidAPI-Key": "fc1e58f82cmsh80f890b0cc58cfep1656acjsnb3d61b0144ed"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    list_for_df.append(pd.json_normalize(response.json()['items'][0]))

  return pd.concat(list_for_df, ignore_index=True)

In [7]:
icao_airport_codes([52.31, 52.37], [13.24, 4.89])

,icao,iata,name,shortName,municipalityName,countryCode,location.lat,location.lon
0,EDDB,BER,"Berlin, Berlin Brandenburg",Brandenburg,Berlin,DE,52.35139,13.493889
1,EHAM,AMS,"Amsterdam, Amsterdam Schiphol",Schiphol,Amsterdam,NL,52.30860,4.763889


In [8]:
import requests

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EHAM/2023-02-09T08:00/2023-02-09T20:00"

querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"true","withLocation":"false"}

headers = {
	"X-RapidAPI-Key": "fc1e58f82cmsh80f890b0cc58cfep1656acjsnb3d61b0144ed",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"arrivals":[{"movement":{"airport":{"icao":"SMJP","iata":"PBM","name":"Zandery"},"scheduledTimeLocal":"2023-02-09 07:20+01:00","actualTimeLocal":"2023-02-09 08:54+01:00","runwayTimeLocal":"2023-02-09 08:54+01:00","scheduledTimeUtc":"2023-02-09 06:20Z","actualTimeUtc":"2023-02-09 07:54Z","runwayTimeUtc":"2023-02-09 07:54Z","scheduledTime":{"utc":"2023-02-09 06:20Z","local":"2023-02-09 07:20+01:00"},"revisedTime":{"utc":"2023-02-09 07:54Z","local":"2023-02-09 08:54+01:00"},"runwayTime":{"utc":"2023-02-09 07:54Z","local":"2023-02-09 08:54+01:00"},"terminal":"4","baggageBelt":"22","runway":"18R","quality":["Basic","Live"]},"number":"PY 994","callSign":"SLM994","status":"Arrived","codeshareStatus":"IsOperator","isCargo":false,"aircraft":{"modeS":"4D2439","model":"Airbus A330-200"},"airline":{"name":"Surinam Airways","iata":"PY","icao":"SLM"}},{"movement":{"airport":{"icao":"LFPG","iata":"CDG","name":"Paris"},"scheduledTimeLocal":"2023-02-09 08:35+01:00","actualTimeLocal":"2023-02-09 08:24+

In [10]:
arrivals=response.json()
arrivals_norm=pd.json_normalize(arrivals["arrivals"],
                                errors="ignore")
arrivals_norm_2=arrivals_norm[["number","status","movement.scheduledTimeLocal"]].copy()
arrivals_norm_2.rename(columns={"number":"Flight_no.",
                                "status":"Status",
                                "movement.scheduledTimeLocal":"Arrival_time"},
                       inplace=True)
arrivals_norm_2

,Flight_no.,Status,Arrival_time
0,PY 994,Arrived,2023-02-09 07:20+01:00
1,AF 1240,Arrived,2023-02-09 08:35+01:00
2,AF 5132,Arrived,2023-02-09 08:20+01:00
3,AF 5597,Arrived,2023-02-09 08:35+01:00
4,AF 8218,Arrived,2023-02-09 08:35+01:00
...,...,...,...
1333,MF 9754,Arrived,2023-02-09 20:00+01:00
1334,VS 6924,Arrived,2023-02-09 20:10+01:00
1335,VS 6950,Arrived,2023-02-09 20:00+01:00
1336,WY 5032,Arrived,2023-02-09 20:10+01:00


In [11]:
from pandas.core.arrays.datetimelike import timedelta
from datetime import datetime, timedelta

dt1=datetime.today()+ timedelta(days=1)
date_plus_1=str(dt1.date())
date_plus_1

'2023-05-23'

In [14]:
def money_arrives(ICAO):

  list_for_df = []

  dt1=datetime.today()+ timedelta(days=1)
  date_plus_1=str(dt1.date())

  for icao in ICAO:
  
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{date_plus_1}T08:00/{date_plus_1}T20:00"
  
    querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"true","withLocation":"false"}

    headers = {
        "X-RapidAPI-Key": "fc1e58f82cmsh80f890b0cc58cfep1656acjsnb3d61b0144ed",
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    #list_for_df.append(pd.json_normalize(response.json()['arrivals']))
    
    arrivals=response.json()
    for i in arrivals:
    
      arrivals_norm=pd.json_normalize(arrivals["arrivals"],
                                errors="ignore")
      
      schedule=arrivals_norm[["number","status","movement.scheduledTimeLocal"]].copy()
      schedule.insert(0,"Airport",icao)
      schedule.rename(columns={"number":"Flight_no.",
                                "status":"Status",
                                "movement.scheduledTimeLocal":"Arrival_time"},
                       inplace=True)
      list_for_df.append(schedule)
      final=pd.concat(list_for_df, ignore_index=True)
  return final


In [15]:
money_arrives(["EDDB","EHAM"])

,Airport,Flight_no.,Status,Arrival_time
0,EDDB,AC 6802,Expected,2023-05-23 08:45+02:00
1,EDDB,AC 9117,Expected,2023-05-23 08:55+02:00
2,EDDB,AY 1431,Expected,2023-05-23 08:05+02:00
3,EDDB,EC 4897,Expected,2023-05-23 08:45+02:00
4,EDDB,EC 5068,Expected,2023-05-23 08:45+02:00
...,...,...,...,...
2108,EHAM,MF 9757,Expected,2023-05-23 20:05+02:00
2109,EHAM,MF 9952,Expected,2023-05-23 20:00+02:00
2110,EHAM,MF 9998,Expected,2023-05-23 20:05+02:00
2111,EHAM,MH 5680,Expected,2023-05-23 20:00+02:00
